<a href="https://colab.research.google.com/github/UtkarshAIITB/IPL-Data-Analysis/blob/main/ML_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [468]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

In [469]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [470]:
ball = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/IPL Score_Analysis/CSV/IPL Ball-by-Ball 2008-2020.csv')
print(ball.shape)
ball.head() 

(193468, 18)


,id,inning,over,ball,batsman,non_striker,bowler,batsman_runs,extra_runs,total_runs,non_boundary,is_wicket,dismissal_kind,player_dismissed,fielder,extras_type,batting_team,bowling_team
0,335982,1,6,5,RT Ponting,BB McCullum,AA Noffke,1,0,1,0,0,NaN,NaN,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore
1,335982,1,6,6,BB McCullum,RT Ponting,AA Noffke,1,0,1,0,0,NaN,NaN,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore
2,335982,1,7,1,BB McCullum,RT Ponting,Z Khan,0,0,0,0,0,NaN,NaN,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore
3,335982,1,7,2,BB McCullum,RT Ponting,Z Khan,1,0,1,0,0,NaN,NaN,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore
4,335982,1,7,3,RT Ponting,BB McCullum,Z Khan,1,0,1,0,0,NaN,NaN,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore


In [471]:
match = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/IPL Score_Analysis/CSV/IPL Matches 2008-2020.csv')
print(match.shape)
match.head() 

(816, 17)


,id,city,date,player_of_match,venue,neutral_venue,team1,team2,toss_winner,toss_decision,winner,result,result_margin,eliminator,method,umpire1,umpire2
0,335982,Bangalore,2008-04-18,BB McCullum,M Chinnaswamy Stadium,0,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,140.0,N,NaN,Asad Rauf,RE Koertzen
1,335983,Chandigarh,2008-04-19,MEK Hussey,"Punjab Cricket Association Stadium, Mohali",0,Kings XI Punjab,Chennai Super Kings,Chennai Super Kings,bat,Chennai Super Kings,runs,33.0,N,NaN,MR Benson,SL Shastri
2,335984,Delhi,2008-04-19,MF Maharoof,Feroz Shah Kotla,0,Delhi Daredevils,Rajasthan Royals,Rajasthan Royals,bat,Delhi Daredevils,wickets,9.0,N,NaN,Aleem Dar,GA Pratapkumar
3,335985,Mumbai,2008-04-20,MV Boucher,Wankhede Stadium,0,Mumbai Indians,Royal Challengers Bangalore,Mumbai Indians,bat,Royal Challengers Bangalore,wickets,5.0,N,NaN,SJ Davis,DJ Harper
4,335986,Kolkata,2008-04-20,DJ Hussey,Eden Gardens,0,Kolkata Knight Riders,Deccan Chargers,Deccan Chargers,bat,Kolkata Knight Riders,wickets,5.0,N,NaN,BF Bowden,K Hariharan


In [472]:
df = pd.merge(match , ball, on='id', how = 'right')
print(df.shape)
df.head()

(193468, 34)


,id,city,date,player_of_match,venue,neutral_venue,team1,team2,toss_winner,toss_decision,winner,result,result_margin,eliminator,method,umpire1,umpire2,inning,over,ball,batsman,non_striker,bowler,batsman_runs,extra_runs,total_runs,non_boundary,is_wicket,dismissal_kind,player_dismissed,fielder,extras_type,batting_team,bowling_team
0,335982,Bangalore,2008-04-18,BB McCullum,M Chinnaswamy Stadium,0,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,140.0,N,NaN,Asad Rauf,RE Koertzen,1,6,5,RT Ponting,BB McCullum,AA Noffke,1,0,1,0,0,NaN,NaN,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore
1,335982,Bangalore,2008-04-18,BB McCullum,M Chinnaswamy Stadium,0,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,140.0,N,NaN,Asad Rauf,RE Koertzen,1,6,6,BB McCullum,RT Ponting,AA Noffke,1,0,1,0,0,NaN,NaN,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore
2,335982,Bangalore,2008-04-18,BB McCullum,M Chinnaswamy Stadium,0,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,140.0,N,NaN,Asad Rauf,RE Koertzen,1,7,1,BB McCullum,RT Ponting,Z Khan,0,0,0,0,0,NaN,NaN,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore
3,335982,Bangalore,2008-04-18,BB McCullum,M Chinnaswamy Stadium,0,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,140.0,N,NaN,Asad Rauf,RE Koertzen,1,7,2,BB McCullum,RT Ponting,Z Khan,1,0,1,0,0,NaN,NaN,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore
4,335982,Bangalore,2008-04-18,BB McCullum,M Chinnaswamy Stadium,0,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,140.0,N,NaN,Asad Rauf,RE Koertzen,1,7,3,RT Ponting,BB McCullum,Z Khan,1,0,1,0,0,NaN,NaN,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore


## Pre-processing

### Converting names of the teams

In [473]:
df = df.replace(['Rising Pune Supergiant' , 'Delhi Daredevils'], ['Rising Pune Supergiants' , 'Delhi Capitals'])

In [474]:
df['year'] = pd.DatetimeIndex(df['date']).year
# data['year'] = pd.DatetimeIndex(data['date']).year
df.head()

,id,city,date,player_of_match,venue,neutral_venue,team1,team2,toss_winner,toss_decision,winner,result,result_margin,eliminator,method,umpire1,umpire2,inning,over,ball,batsman,non_striker,bowler,batsman_runs,extra_runs,total_runs,non_boundary,is_wicket,dismissal_kind,player_dismissed,fielder,extras_type,batting_team,bowling_team,year
0,335982,Bangalore,2008-04-18,BB McCullum,M Chinnaswamy Stadium,0,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,140.0,N,NaN,Asad Rauf,RE Koertzen,1,6,5,RT Ponting,BB McCullum,AA Noffke,1,0,1,0,0,NaN,NaN,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore,2008
1,335982,Bangalore,2008-04-18,BB McCullum,M Chinnaswamy Stadium,0,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,140.0,N,NaN,Asad Rauf,RE Koertzen,1,6,6,BB McCullum,RT Ponting,AA Noffke,1,0,1,0,0,NaN,NaN,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore,2008
2,335982,Bangalore,2008-04-18,BB McCullum,M Chinnaswamy Stadium,0,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,140.0,N,NaN,Asad Rauf,RE Koertzen,1,7,1,BB McCullum,RT Ponting,Z Khan,0,0,0,0,0,NaN,NaN,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore,2008
3,335982,Bangalore,2008-04-18,BB McCullum,M Chinnaswamy Stadium,0,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,140.0,N,NaN,Asad Rauf,RE Koertzen,1,7,2,BB McCullum,RT Ponting,Z Khan,1,0,1,0,0,NaN,NaN,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore,2008
4,335982,Bangalore,2008-04-18,BB McCullum,M Chinnaswamy Stadium,0,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,140.0,N,NaN,Asad Rauf,RE Koertzen,1,7,3,RT Ponting,BB McCullum,Z Khan,1,0,1,0,0,NaN,NaN,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore,2008


Dropping unnecessary columns

In [475]:
columns = ['neutral_venue', 'result', 'result_margin', 'umpire1', 'umpire2', 'batsman', 'non_striker', 'bowler', 'non_boundary', 'dismissal_kind',
           'player_dismissed', 'fielder', 'extras_type', 'player_of_match', 'venue', 'city']
df_run = df.drop(columns = columns)
df_run.head()

,id,date,team1,team2,toss_winner,toss_decision,winner,eliminator,method,inning,over,ball,batsman_runs,extra_runs,total_runs,is_wicket,batting_team,bowling_team,year
0,335982,2008-04-18,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,N,NaN,1,6,5,1,0,1,0,Kolkata Knight Riders,Royal Challengers Bangalore,2008
1,335982,2008-04-18,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,N,NaN,1,6,6,1,0,1,0,Kolkata Knight Riders,Royal Challengers Bangalore,2008
2,335982,2008-04-18,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,N,NaN,1,7,1,0,0,0,0,Kolkata Knight Riders,Royal Challengers Bangalore,2008
3,335982,2008-04-18,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,N,NaN,1,7,2,1,0,1,0,Kolkata Knight Riders,Royal Challengers Bangalore,2008
4,335982,2008-04-18,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,N,NaN,1,7,3,1,0,1,0,Kolkata Knight Riders,Royal Challengers Bangalore,2008


In [476]:
df_run['method']  = df_run['method'].fillna('Normal')
df_run.head()

,id,date,team1,team2,toss_winner,toss_decision,winner,eliminator,method,inning,over,ball,batsman_runs,extra_runs,total_runs,is_wicket,batting_team,bowling_team,year
0,335982,2008-04-18,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,N,Normal,1,6,5,1,0,1,0,Kolkata Knight Riders,Royal Challengers Bangalore,2008
1,335982,2008-04-18,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,N,Normal,1,6,6,1,0,1,0,Kolkata Knight Riders,Royal Challengers Bangalore,2008
2,335982,2008-04-18,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,N,Normal,1,7,1,0,0,0,0,Kolkata Knight Riders,Royal Challengers Bangalore,2008
3,335982,2008-04-18,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,N,Normal,1,7,2,1,0,1,0,Kolkata Knight Riders,Royal Challengers Bangalore,2008
4,335982,2008-04-18,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,N,Normal,1,7,3,1,0,1,0,Kolkata Knight Riders,Royal Challengers Bangalore,2008


In [477]:
winner_null = df_run[df_run['winner'].isna()]
df_run = df_run.dropna()

In [478]:
df_sort = df_run.sort_values(by = ['id', 'inning', 'over', 'ball'])
df_sort = df_sort.reset_index()
df_sort = df_sort.drop(columns = 'index')
df_sort.head()

,id,date,team1,team2,toss_winner,toss_decision,winner,eliminator,method,inning,over,ball,batsman_runs,extra_runs,total_runs,is_wicket,batting_team,bowling_team,year
0,335982,2008-04-18,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,N,Normal,1,0,1,0,1,1,0,Kolkata Knight Riders,Royal Challengers Bangalore,2008
1,335982,2008-04-18,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,N,Normal,1,0,2,0,0,0,0,Kolkata Knight Riders,Royal Challengers Bangalore,2008
2,335982,2008-04-18,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,N,Normal,1,0,3,0,1,1,0,Kolkata Knight Riders,Royal Challengers Bangalore,2008
3,335982,2008-04-18,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,N,Normal,1,0,4,0,0,0,0,Kolkata Knight Riders,Royal Challengers Bangalore,2008
4,335982,2008-04-18,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,N,Normal,1,0,5,0,0,0,0,Kolkata Knight Riders,Royal Challengers Bangalore,2008


### One Hot encoding of toss decision, method and eliminator

In [479]:
dummy_td = pd.get_dummies(df_sort.toss_decision)
df_enc = pd.concat([df_sort, dummy_td], axis = 'columns')                     #encoded toss decisions - field or bat
df_enc = df_enc.drop(columns = ['toss_decision', 'bat'])
# df_enc.head()
dummy_td = pd.get_dummies(df_sort.method)
df_enc = pd.concat([df_enc, dummy_td], axis = 'columns')
df_enc = df_enc.drop(columns = ['method', 'Normal'])                          #encoded method - normal or D/L
# df_enc.head()
dummy_td = pd.get_dummies(df_sort.eliminator)
dummy_td.rename(columns = {'Y': 'eliminator?' }, inplace = True )
df_enc = pd.concat([df_enc, dummy_td], axis = 'columns')
df_enc = df_enc.drop(columns = ['eliminator', 'N'])  
df_enc.head()

,id,date,team1,team2,toss_winner,winner,inning,over,ball,batsman_runs,extra_runs,total_runs,is_wicket,batting_team,bowling_team,year,field,D/L,eliminator?
0,335982,2008-04-18,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,Kolkata Knight Riders,1,0,1,0,1,1,0,Kolkata Knight Riders,Royal Challengers Bangalore,2008,1,0,0
1,335982,2008-04-18,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,Kolkata Knight Riders,1,0,2,0,0,0,0,Kolkata Knight Riders,Royal Challengers Bangalore,2008,1,0,0
2,335982,2008-04-18,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,Kolkata Knight Riders,1,0,3,0,1,1,0,Kolkata Knight Riders,Royal Challengers Bangalore,2008,1,0,0
3,335982,2008-04-18,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,Kolkata Knight Riders,1,0,4,0,0,0,0,Kolkata Knight Riders,Royal Challengers Bangalore,2008,1,0,0
4,335982,2008-04-18,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,Kolkata Knight Riders,1,0,5,0,0,0,0,Kolkata Knight Riders,Royal Challengers Bangalore,2008,1,0,0


In [480]:
df_enc['inning_bat'] = df_enc['id'].astype(str)+(df_enc['batting_team']).astype(str)
df_enc.head()

,id,date,team1,team2,toss_winner,winner,inning,over,ball,batsman_runs,extra_runs,total_runs,is_wicket,batting_team,bowling_team,year,field,D/L,eliminator?,inning_bat
0,335982,2008-04-18,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,Kolkata Knight Riders,1,0,1,0,1,1,0,Kolkata Knight Riders,Royal Challengers Bangalore,2008,1,0,0,335982Kolkata Knight Riders
1,335982,2008-04-18,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,Kolkata Knight Riders,1,0,2,0,0,0,0,Kolkata Knight Riders,Royal Challengers Bangalore,2008,1,0,0,335982Kolkata Knight Riders
2,335982,2008-04-18,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,Kolkata Knight Riders,1,0,3,0,1,1,0,Kolkata Knight Riders,Royal Challengers Bangalore,2008,1,0,0,335982Kolkata Knight Riders
3,335982,2008-04-18,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,Kolkata Knight Riders,1,0,4,0,0,0,0,Kolkata Knight Riders,Royal Challengers Bangalore,2008,1,0,0,335982Kolkata Knight Riders
4,335982,2008-04-18,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,Kolkata Knight Riders,1,0,5,0,0,0,0,Kolkata Knight Riders,Royal Challengers Bangalore,2008,1,0,0,335982Kolkata Knight Riders


In [481]:
list1 = df_enc['inning_bat'].unique()
df_copy = df_enc.copy()
df_copy = df_copy.groupby(by = ['inning_bat']).sum()['total_runs'].reset_index()
data = pd.merge(left=df_enc, right=df_copy, on='inning_bat', how='right')
print(data.shape)
data.head()

(193096, 21)


,id,date,team1,team2,toss_winner,winner,inning,over,ball,batsman_runs,extra_runs,total_runs_x,is_wicket,batting_team,bowling_team,year,field,D/L,eliminator?,inning_bat,total_runs_y
0,1082591,2017-04-05,Sunrisers Hyderabad,Royal Challengers Bangalore,Royal Challengers Bangalore,Sunrisers Hyderabad,2,0,1,1,0,1,0,Royal Challengers Bangalore,Sunrisers Hyderabad,2017,1,0,0,1082591Royal Challengers Bangalore,172
1,1082591,2017-04-05,Sunrisers Hyderabad,Royal Challengers Bangalore,Royal Challengers Bangalore,Sunrisers Hyderabad,2,0,2,0,0,0,0,Royal Challengers Bangalore,Sunrisers Hyderabad,2017,1,0,0,1082591Royal Challengers Bangalore,172
2,1082591,2017-04-05,Sunrisers Hyderabad,Royal Challengers Bangalore,Royal Challengers Bangalore,Sunrisers Hyderabad,2,0,3,0,0,0,0,Royal Challengers Bangalore,Sunrisers Hyderabad,2017,1,0,0,1082591Royal Challengers Bangalore,172
3,1082591,2017-04-05,Sunrisers Hyderabad,Royal Challengers Bangalore,Royal Challengers Bangalore,Sunrisers Hyderabad,2,0,4,2,0,2,0,Royal Challengers Bangalore,Sunrisers Hyderabad,2017,1,0,0,1082591Royal Challengers Bangalore,172
4,1082591,2017-04-05,Sunrisers Hyderabad,Royal Challengers Bangalore,Royal Challengers Bangalore,Sunrisers Hyderabad,2,0,5,4,0,4,0,Royal Challengers Bangalore,Sunrisers Hyderabad,2017,1,0,0,1082591Royal Challengers Bangalore,172


In [482]:
data['over'] = data['over']+ (data['ball'])/10
data.drop(columns = ['ball'], inplace = True)
print(data.shape)
data.head()

(193096, 20)


,id,date,team1,team2,toss_winner,winner,inning,over,batsman_runs,extra_runs,total_runs_x,is_wicket,batting_team,bowling_team,year,field,D/L,eliminator?,inning_bat,total_runs_y
0,1082591,2017-04-05,Sunrisers Hyderabad,Royal Challengers Bangalore,Royal Challengers Bangalore,Sunrisers Hyderabad,2,0.1,1,0,1,0,Royal Challengers Bangalore,Sunrisers Hyderabad,2017,1,0,0,1082591Royal Challengers Bangalore,172
1,1082591,2017-04-05,Sunrisers Hyderabad,Royal Challengers Bangalore,Royal Challengers Bangalore,Sunrisers Hyderabad,2,0.2,0,0,0,0,Royal Challengers Bangalore,Sunrisers Hyderabad,2017,1,0,0,1082591Royal Challengers Bangalore,172
2,1082591,2017-04-05,Sunrisers Hyderabad,Royal Challengers Bangalore,Royal Challengers Bangalore,Sunrisers Hyderabad,2,0.3,0,0,0,0,Royal Challengers Bangalore,Sunrisers Hyderabad,2017,1,0,0,1082591Royal Challengers Bangalore,172
3,1082591,2017-04-05,Sunrisers Hyderabad,Royal Challengers Bangalore,Royal Challengers Bangalore,Sunrisers Hyderabad,2,0.4,2,0,2,0,Royal Challengers Bangalore,Sunrisers Hyderabad,2017,1,0,0,1082591Royal Challengers Bangalore,172
4,1082591,2017-04-05,Sunrisers Hyderabad,Royal Challengers Bangalore,Royal Challengers Bangalore,Sunrisers Hyderabad,2,0.5,4,0,4,0,Royal Challengers Bangalore,Sunrisers Hyderabad,2017,1,0,0,1082591Royal Challengers Bangalore,172


### Finding total runs

In [483]:
df_final = pd.DataFrame()
for bat in list1:
  inning  = data[data['inning_bat']==bat].copy()
  # inning.head()
  cum_total  = inning.loc[:,'total_runs_x'].cumsum(axis = 0)
  inning['cum_total'] = cum_total
  cum_wicket = inning.loc[:,'is_wicket'].cumsum(axis = 0)
  inning['cum_wicket'] = cum_wicket
  # inning.head()
  df_final = df_final.append(inning, ignore_index=True)

In [484]:
df_final.head()

,id,date,team1,team2,toss_winner,winner,inning,over,batsman_runs,extra_runs,total_runs_x,is_wicket,batting_team,bowling_team,year,field,D/L,eliminator?,inning_bat,total_runs_y,cum_total,cum_wicket
0,335982,2008-04-18,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,Kolkata Knight Riders,1,0.1,0,1,1,0,Kolkata Knight Riders,Royal Challengers Bangalore,2008,1,0,0,335982Kolkata Knight Riders,222,1,0
1,335982,2008-04-18,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,Kolkata Knight Riders,1,0.2,0,0,0,0,Kolkata Knight Riders,Royal Challengers Bangalore,2008,1,0,0,335982Kolkata Knight Riders,222,1,0
2,335982,2008-04-18,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,Kolkata Knight Riders,1,0.3,0,1,1,0,Kolkata Knight Riders,Royal Challengers Bangalore,2008,1,0,0,335982Kolkata Knight Riders,222,2,0
3,335982,2008-04-18,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,Kolkata Knight Riders,1,0.4,0,0,0,0,Kolkata Knight Riders,Royal Challengers Bangalore,2008,1,0,0,335982Kolkata Knight Riders,222,2,0
4,335982,2008-04-18,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,Kolkata Knight Riders,1,0.5,0,0,0,0,Kolkata Knight Riders,Royal Challengers Bangalore,2008,1,0,0,335982Kolkata Knight Riders,222,2,0


### Label encoding
Teams are label encoded in order of their strength analysed from EDA

In [485]:
team_dict = {'Mumbai Indians':13, 'Chennai Super Kings':12, 'Rajasthan Royals':11, 'Rising Pune Supergiants':10 ,'Kolkata Knight Riders':9, 'Kings XI Punjab':8,
             'Sunrisers Hyderabad':7, 'Delhi Capitals':6, 'Royal Challengers Bangalore':5, 'Gujarat Lions':4, 'Deccan Chargers':3,
             'Pune Warriors':2, 'Kochi Tuskers Kerala':1
             }

In [486]:
bore = df_final.copy()
bore['team1'] = bore.team1.map(team_dict)
bore['team2'] = bore.team2.map(team_dict)
bore['toss_winner'] = bore.toss_winner.map(team_dict)
bore['winner'] = bore.winner.map(team_dict)
bore['batting_team'] = bore.batting_team.map(team_dict)
bore['bowling_team'] = bore.bowling_team.map(team_dict)
bore.head()

,id,date,team1,team2,toss_winner,winner,inning,over,batsman_runs,extra_runs,total_runs_x,is_wicket,batting_team,bowling_team,year,field,D/L,eliminator?,inning_bat,total_runs_y,cum_total,cum_wicket
0,335982,2008-04-18,5,9,5,9,1,0.1,0,1,1,0,9,5,2008,1,0,0,335982Kolkata Knight Riders,222,1,0
1,335982,2008-04-18,5,9,5,9,1,0.2,0,0,0,0,9,5,2008,1,0,0,335982Kolkata Knight Riders,222,1,0
2,335982,2008-04-18,5,9,5,9,1,0.3,0,1,1,0,9,5,2008,1,0,0,335982Kolkata Knight Riders,222,2,0
3,335982,2008-04-18,5,9,5,9,1,0.4,0,0,0,0,9,5,2008,1,0,0,335982Kolkata Knight Riders,222,2,0
4,335982,2008-04-18,5,9,5,9,1,0.5,0,0,0,0,9,5,2008,1,0,0,335982Kolkata Knight Riders,222,2,0


In [487]:
bore.drop(columns = ['extra_runs', 'inning_bat', 'batsman_runs', 'total_runs_x', 'is_wicket'], inplace = True)
bore.head()

,id,date,team1,team2,toss_winner,winner,inning,over,batting_team,bowling_team,year,field,D/L,eliminator?,total_runs_y,cum_total,cum_wicket
0,335982,2008-04-18,5,9,5,9,1,0.1,9,5,2008,1,0,0,222,1,0
1,335982,2008-04-18,5,9,5,9,1,0.2,9,5,2008,1,0,0,222,1,0
2,335982,2008-04-18,5,9,5,9,1,0.3,9,5,2008,1,0,0,222,2,0
3,335982,2008-04-18,5,9,5,9,1,0.4,9,5,2008,1,0,0,222,2,0
4,335982,2008-04-18,5,9,5,9,1,0.5,9,5,2008,1,0,0,222,2,0


In [488]:
test_set = bore[bore.year == 2019]
test_set.head()

,id,date,team1,team2,toss_winner,winner,inning,over,batting_team,bowling_team,year,field,D/L,eliminator?,total_runs_y,cum_total,cum_wicket
164344,1175356,2019-03-23,12,5,12,12,1,0.1,5,12,2019,1,0,0,70,1,0
164345,1175356,2019-03-23,12,5,12,12,1,0.2,5,12,2019,1,0,0,70,1,0
164346,1175356,2019-03-23,12,5,12,12,1,0.3,5,12,2019,1,0,0,70,1,0
164347,1175356,2019-03-23,12,5,12,12,1,0.4,5,12,2019,1,0,0,70,1,0
164348,1175356,2019-03-23,12,5,12,12,1,0.5,5,12,2019,1,0,0,70,5,0


test_set contains complete IPL data for the year 2019.
We will train the model on the data from the year 2008-2020

In [489]:
bore.drop(bore[bore.year == 2019].index, inplace = True)
print(bore.shape)

(178854, 17)


In [490]:
demo = bore[bore.year == 2019]
demo.head()
#as expected demo is empty.

,id,date,team1,team2,toss_winner,winner,inning,over,batting_team,bowling_team,year,field,D/L,eliminator?,total_runs_y,cum_total,cum_wicket


## Accuracy Model

In [491]:
def custom_accuracy(y_test,y_pred,thresold):
    right = 0
    l = len(y_pred)
    for i in range(0,l):
        if(abs(y_pred[i]-y_test[i]) <= thresold):
            right += 1
    return ((right/l)*100)

In [492]:
go = bore.copy()

## Total Runs Function

In [493]:
def new_strat(y_pred, y_test):
  sample = pd.DataFrame(y_pred,columns=['Predict'])
  sample['Actual']=y_test
  ar1 = sample['Actual'].unique()
  results = pd.DataFrame()
  for items in ar1:
    # print(items)
    try2 = sample[sample.Actual == items]
    result = (try2.sum())/try2.shape[0]
    # print(result)
    res = result.values.reshape(1,2)
    # print(res)
    results = results.append(pd.DataFrame(res))

  results.rename(columns = {0: 'Predict',
                            1: 'Actual'
                            }, inplace = True)
  results.reset_index(drop = True)
  accu = (custom_accuracy(results['Actual'].ravel(),results['Predict'].ravel(),12))
  return(accu)

# Runs prediction - Primary Models

In [494]:
y1 = go['winner']
y2 = go['total_runs_y'] 
X = go.drop(columns = ['id', 'date', 'winner', 'total_runs_y','year'])

In [495]:
X.head()

,team1,team2,toss_winner,inning,over,batting_team,bowling_team,field,D/L,eliminator?,cum_total,cum_wicket
0,5,9,5,1,0.1,9,5,1,0,0,1,0
1,5,9,5,1,0.2,9,5,1,0,0,1,0
2,5,9,5,1,0.3,9,5,1,0,0,2,0
3,5,9,5,1,0.4,9,5,1,0,0,2,0
4,5,9,5,1,0.5,9,5,1,0,0,2,0


In [496]:
test = test_set.copy()
y1_test = test['winner']
y2_test = test['total_runs_y']
X_test = test.drop(columns = ['id', 'date', 'winner', 'total_runs_y','year'])

In [497]:
X_test.reset_index().drop(columns = ['index']).head()

,team1,team2,toss_winner,inning,over,batting_team,bowling_team,field,D/L,eliminator?,cum_total,cum_wicket
0,12,5,12,1,0.1,5,12,1,0,0,1,0
1,12,5,12,1,0.2,5,12,1,0,0,1,0
2,12,5,12,1,0.3,5,12,1,0,0,1,0
3,12,5,12,1,0.4,5,12,1,0,0,1,0
4,12,5,12,1,0.5,5,12,1,0,0,5,0


In [498]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X = sc.fit_transform(X)

## Complete Data

### Random Forest

In [499]:
%%time
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()
rf.fit(X, y2)
y_pred = rf.predict(X_test)
y2_test = y2_test.ravel()
print("Custom accuracy: {:.3f} %".format(custom_accuracy(y2_test,y_pred,10)))

Custom accuracy: 2.493 %
CPU times: user 1min 6s, sys: 657 ms, total: 1min 7s
Wall time: 1min 7s


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning:

X has feature names, but RandomForestRegressor was fitted without feature names



### Linear Regression

In [500]:
from sklearn.linear_model import LinearRegression

In [501]:
lr = LinearRegression()
lr.fit(X, y2)
y_pred = lr.predict(X_test)
print("Custom accuracy: {:.3f} %".format(custom_accuracy(y2_test,y_pred,10)))

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning:

X has feature names, but LinearRegression was fitted without feature names



Custom accuracy: 0.000 %


As it can be seen clearly, both the regressors dint work. Since the data is too much, so its highly overfitting. So lets try out training the model on a particular ball.

## Particular ball

So since overall our model failed, lets train the model ball by ball and check in which range of overs the model performs best. By domain knowledge, it mustn't be good for first 6-7 overs since that has very little to do with teams total score

In [502]:
def accurate(ball, threshold):
  bb = bore[bore.over == ball]
  bb_test = test_set[test_set.over == ball]
  y1 = bb['winner']
  y2 = bb['total_runs_y']
  X = bb.drop(columns = ['winner', 'total_runs_y', 'year','id', 'date']).reset_index(drop = True)
  y1_test = bb_test['winner']
  y2_test = bb_test['total_runs_y']
  X_test = bb_test.drop(columns = ['winner', 'total_runs_y', 'year','id', 'date']).reset_index(drop = True)
  X_trans = sc.fit_transform(X)
  X_trans_test = sc.fit_transform(X_test)
  svr.fit(X_trans,y2)
  y_pred = svr.predict(X_trans_test)
  y2_test = y2_test.ravel()
  print('Accuracy in ball '+str(ball)+' is ' + str(custom_accuracy(y2_test, y_pred, threshold)) + '% in SVR')
  rf.fit(X_trans,y2)
  y_pred = rf.predict(X_trans_test)
  print('Accuracy in ball '+str(ball)+' is ' + str(custom_accuracy(y2_test, y_pred, threshold)) + '% in RF')

In [503]:
list3 = bore['over'].unique()
list4 = test['over'].unique()
set3 = set(list3)
set4 = set(list4)
for ball in (set3 & set4):
  accurate(ball,12)

Accuracy in ball 0.5 is 28.8135593220339% in SVR
Accuracy in ball 0.5 is 31.35593220338983% in RF
Accuracy in ball 1.1 is 30.508474576271187% in SVR
Accuracy in ball 1.1 is 27.966101694915253% in RF
Accuracy in ball 1.4 is 30.508474576271187% in SVR
Accuracy in ball 1.4 is 27.966101694915253% in RF
Accuracy in ball 1.3 is 30.508474576271187% in SVR
Accuracy in ball 1.3 is 26.27118644067797% in RF
Accuracy in ball 1.2 is 33.05084745762712% in SVR
Accuracy in ball 1.2 is 32.20338983050847% in RF
Accuracy in ball 1.5 is 29.66101694915254% in SVR
Accuracy in ball 1.5 is 28.8135593220339% in RF
Accuracy in ball 1.6 is 28.8135593220339% in SVR
Accuracy in ball 1.6 is 23.728813559322035% in RF
Accuracy in ball 2.1 is 31.35593220338983% in SVR
Accuracy in ball 2.1 is 28.8135593220339% in RF
Accuracy in ball 2.5 is 33.05084745762712% in SVR
Accuracy in ball 2.5 is 33.89830508474576% in RF
Accuracy in ball 4.1 is 36.440677966101696% in SVR
Accuracy in ball 4.1 is 29.66101694915254% in RF
Accurac

General observation : SVR has higher accuracy for overs greater than 7 while for overs less than 7 RF has a higher accuracy. <br>
So taking a glance at the result above we will run the run prediction model from overs 11-16.

## Overs 11-16

### New Variables 

t contains the data for the year we are choosing as CV i.e. it consists of 2019 data. <br> 
y1test and y2test  = 2019 <br> <br>
new contains the data for the years 2008 - 2020 except the data which is in t.
<br>
y1train and y2 train = 2008-2020<br><br>
tests contain 2021 data <br>
y1t and y2t = 2021

In [504]:
new = go.copy()
t = test_set.copy()

In [505]:
t.to_csv('/content/drive/MyDrive/Colab Notebooks/IPL Score_Analysis/CSV/t.csv')
new.to_csv('/content/drive/MyDrive/Colab Notebooks/IPL Score_Analysis/CSV/new.csv')

In [506]:
tnew = t.drop(t[(t.over <= 11.0) | (t.over >=16.0)].index)
tnew.reset_index(drop = True)
id_test = tnew['id'].reset_index(drop = True)
id_test = id_test.to_frame().reset_index(drop=True)
y1test = tnew['total_runs_y']
y2test = tnew['winner']
tnew.drop(columns = ['id', 'date', 'year', 'total_runs_y', 'winner'], inplace = True)
tnew.reset_index(drop=True).head()

,team1,team2,toss_winner,inning,over,batting_team,bowling_team,field,D/L,eliminator?,cum_total,cum_wicket
0,12,5,12,1,11.1,5,12,1,0,0,53,7
1,12,5,12,1,11.2,5,12,1,0,0,53,7
2,12,5,12,1,11.3,5,12,1,0,0,53,7
3,12,5,12,1,11.4,5,12,1,0,0,54,7
4,12,5,12,1,11.5,5,12,1,0,0,55,7


In [507]:
print(tnew.shape)

(3640, 12)


In [508]:
xnew = new.drop(new[(new.over <= 11.0) | (new.over >=16.0)].index)
xnew.reset_index(drop = True)
y1train = xnew['total_runs_y']
y2train = xnew['winner']
id_train = xnew['id']
xnew.drop(columns = ['id', 'date', 'year', 'total_runs_y', 'winner'], inplace = True)
xnew.reset_index(drop=True).head()

,team1,team2,toss_winner,inning,over,batting_team,bowling_team,field,D/L,eliminator?,cum_total,cum_wicket
0,5,9,5,1,11.1,9,5,1,0,0,102,1
1,5,9,5,1,11.2,9,5,1,0,0,108,1
2,5,9,5,1,11.3,9,5,1,0,0,110,1
3,5,9,5,1,11.4,9,5,1,0,0,111,1
4,5,9,5,1,11.5,9,5,1,0,0,111,1


In [509]:
print(xnew.shape)

(44764, 12)


In [510]:
tests = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/IPL Score_Analysis/CSV/testset.csv')
tests.drop(columns = ['Unnamed: 0'], inplace = True)
tt_2021 = tests['id'].reset_index(drop = True)
tt_2021 = tt_2021.to_frame().reset_index(drop = True)
# tests.head()
sets = tests.drop(tests[(tests.over <= 11.0) | (tests.over >=16.0)].index)
sets.reset_index(drop = True)
yt1 = sets['total_runs_y']
yt2 = sets['winner']
sets.drop(columns = ['id', 'date', 'total_runs_y', 'winner'], inplace = True)
sets.reset_index(drop=True).head()

,team1,team2,toss_winner,inning,over,batting_team,bowling_team,field,D/L,eliminator?,cum_total,cum_wicket
0,12,5,5,1,11.1,12,5,1,0,0,95,2
1,12,5,5,1,11.2,12,5,1,0,0,95,2
2,12,5,5,1,11.3,12,5,1,0,0,96,2
3,12,5,5,1,11.4,12,5,1,0,0,96,2
4,12,5,5,1,11.5,12,5,1,0,0,97,2


In [511]:
xf = xnew.drop(columns = ['D/L', 'eliminator?'])
tf = tnew.drop(columns = ['D/L', 'eliminator?'])
xs = sets.drop(columns = ['D/L', 'eliminator?'])

### Ravel and scaler transform

In [512]:
yt1 = yt1.ravel()
yt2 = yt2.ravel()
y1test = y1test.ravel()
y2test = y2test.ravel()
y1train = y1train.ravel()
y2train = y2train.ravel()

In [513]:
# with D/L and eliminator
xnewt = sc.fit_transform(xnew) 
tnewt = sc.fit_transform(tnew)
setst = sc.fit_transform(sets)

# without D/L and eliminator
xft = sc.fit_transform(xf) 
tft = sc.fit_transform(tf)
xs = sc.fit_transform(xs)

### Random Forest Regressor

In [514]:
%%time
rf.fit(xnewt, y1train)
y_pred = rf.predict(tnewt)
print('Accuracy: ' + str(custom_accuracy(y1test, y_pred, 12)))

Accuracy: 46.97802197802198
CPU times: user 14 s, sys: 37.8 ms, total: 14.1 s
Wall time: 14 s


In [515]:
rf.fit(xft, y1train)
y_pred = rf.predict(tft)
print('Accuracy without D/L & eliminator: ' + str(custom_accuracy(y1test, y_pred, 12)))

Accuracy without D/L & eliminator: 46.785714285714285


Accuracy is almost same in both cases in RF.

### SVR

In [620]:
from sklearn.svm import SVR

In [621]:
%%time
svr = SVR(C=3)
svr.fit(xnewt, y1train)
y_pred = svr.predict(tnewt)
print('Accuracy: ' + str(custom_accuracy(y1test, y_pred, 12)))

Accuracy: 51.34615384615384
CPU times: user 2min 29s, sys: 144 ms, total: 2min 29s
Wall time: 2min 28s


Now we will adapt a new strategy for total runs that is take average of the total runs of all balls and make a final prediction for that match and then take out the accuracy score.

In [622]:
# CV
new_strat(y_pred, y1test)

60.273972602739725

In [623]:
# test
y_pred = svr.predict(setst)
new_strat(y_pred, yt1)

57.14285714285714

Lets check if overfitting is an issue by removing some more not so important columns.

In [616]:
svr.fit(xft, y1train)
y_pred = svr.predict(tft)
print('Accuracy without D/L & eliminator: ' + str(custom_accuracy(y1test, y_pred, 12)))

Accuracy without D/L & eliminator: 50.3021978021978


Clearly, the accuracy decreased by 0.7% which isn't much but still we shouldn't remove those two columns therefore.

In [617]:
# CV
new_strat(y_pred, y1test)

58.9041095890411

In [618]:
# test
y_pred = svr.predict(xs)
new_strat(y_pred, yt1)

57.14285714285714

In [619]:
SVR().get_params().keys()

dict_keys(['C', 'cache_size', 'coef0', 'degree', 'epsilon', 'gamma', 'kernel', 'max_iter', 'shrinking', 'tol', 'verbose'])

In [625]:
from sklearn.model_selection import GridSearchCV
param_grid_svr = {
              "C": [3,4,6,8,10,20,30,40,50,100],
              "kernel" : ['rbf']           
             }         
gr_svr = GridSearchCV(SVR(), param_grid_svr, cv=5, verbose=2)
gr_svr.fit(xnewt, y1train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV] END ....................................C=3, kernel=rbf; total time= 1.8min
[CV] END ....................................C=3, kernel=rbf; total time= 1.8min
[CV] END ....................................C=3, kernel=rbf; total time= 1.8min
[CV] END ....................................C=3, kernel=rbf; total time= 1.8min
[CV] END ....................................C=3, kernel=rbf; total time= 1.8min
[CV] END ....................................C=4, kernel=rbf; total time= 1.8min
[CV] END ....................................C=4, kernel=rbf; total time= 1.9min
[CV] END ....................................C=4, kernel=rbf; total time= 1.8min
[CV] END ....................................C=4, kernel=rbf; total time= 1.8min
[CV] END ....................................C=4, kernel=rbf; total time= 1.9min
[CV] END ....................................C=6, kernel=rbf; total time= 1.8min
[CV] END ....................................C=6

GridSearchCV(cv=5, estimator=SVR(),
             param_grid={'C': [3, 4, 6, 8, 10, 20, 30, 40, 50, 100],
                         'kernel': ['rbf']},
             verbose=2)

In [626]:
gr_svr.best_params_

{'C': 3, 'kernel': 'rbf'}

In [545]:
# def accuracy_winner(y_test, y_pred):
#   right = 0
#   l = len(y_pred)
#   for i in range(0,1):
#     if(abs(y_pred[i]-y_test[i]) <= 0.5):
#       right +=1
#   return ((right/l)*100)

# Winner Prediction

Since there are basically 13 teams, it is also actually possible to apply a regressor which will give an output from 0 to 13. And we will take the floor of that number as the class it belongs to and then compare the accuracy in the same way we did for total runs by creating a function that computes the average.

## Winner function

In [559]:
def predict_winner(id, y_pred, y_test):
  win = pd.DataFrame(y_pred, columns = ['Predict'])
  win['Actual'] = y_test
  frames = [id_test, win]
  final = pd.concat(frames, axis = 1)

  ar1 = final['id'].unique()
  winner = pd.DataFrame()

  for items in ar1:
    try2 = final[final.id == items]
    # try2.drop(columns = 'id', inplace = True)
    hapi = try2.drop(columns = ['id']).reset_index(drop=True)
    result = (hapi.sum())/hapi.shape[0]
    res = result.values.reshape(1,2)
    winner = winner.append(pd.DataFrame(res))

  winner.rename(columns = {0: 'Predict',
                          1: 'Actual'
                          }, inplace = True)
  winner.reset_index(drop = True)

  winner['diff'] = abs(winner['Actual'] - winner['Predict'])
  to = winner.shape[0]
  final_words = ( ((winner[(winner['diff'])<1].shape[0])/to)*100 )
  return(final_words)

## Regression

### SVR

In [560]:
svr = SVR()
svr.fit(xft, y2train)

# cv
y_pred = svr.predict(tft)
predict_winner(id_test, y_pred, y2test)

45.76271186440678

Test set winner prediction

In [564]:
y_pred = svr.predict(xs)
predict_winner(tt_2021, y_pred, yt2)

89.83050847457628

In [576]:
svr.fit(xnewt, y2train)
y_pred = svr.predict(tnewt)
predict_winner(id_test, y_pred, y2test)

45.76271186440678

In [577]:
y_pred = svr.predict(setst)
predict_winner(id_test, y_pred, yt2)

88.13559322033898

###RF

In [572]:
rf = RandomForestRegressor()
rf.fit(xft, y2train)
y_pred = rf.predict(tnewt)
predict_winner(id_test, y_pred, y2test)

47.45762711864407

Test set winner prediction

In [575]:
y_pred = rf.predict(xs)
predict_winner(tt_2021, y_pred, yt2)

89.83050847457628

## Classifier

### RFC

In [580]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [581]:
clf = RandomForestClassifier()
clf.fit(xnewt, y2train)
y_pred = clf.predict(tnewt)
print(accuracy_score(y2test,y_pred))

0.34697802197802197


### SVC

In [583]:
from sklearn.svm import SVC

In [598]:
svc = SVC(C=0.05)
svc.fit(xnewt, y2train)
y_pred = svc.predict(tnewt)
print(accuracy_score(y2test,y_pred))

0.4967032967032967


In [599]:
y_pred = svc.predict(setst)
predict_winner(id_test, y_pred, yt2)

81.35593220338984